# Comprehensive Data Analysis - Test Dataset

This notebook provides a thorough exploratory data analysis of the test.csv dataset, focusing on:
- Data structure and basic information
- Missing values analysis
- Duplicate detection
- Statistical summaries
- Test vs Training data comparison (matching analyze_data.py)
- Anchor weather analysis
- Data visualization
- Column-wise analysis
- Understanding the prediction task structure

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import os
warnings.filterwarnings('ignore')

# Create graphs directory if it doesn't exist
graphs_dir = 'graphs'
if not os.path.exists(graphs_dir):
    os.makedirs(graphs_dir)
    print(f"Created directory: {graphs_dir}")
else:
    print(f"Directory {graphs_dir} already exists")

# Create test graphs directory if it doesn't exist
test_graphs_dir = f'{graphs_dir}/test_graphs_dir'
if not os.path.exists(test_graphs_dir):
    os.makedirs(test_graphs_dir)
    print(f"Created directory: {test_graphs_dir}")
else:
    print(f"Directory {test_graphs_dir} already exists")

# Set plotting style
plt.style.use('default')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Data Loading and Initial Inspection

In [ ]:
# Load the test dataset
test_df = pd.read_csv('../data/test.csv')

print("=== TEST DATASET ===")
print(f"Shape: {test_df.shape}")
print(f"Columns: {list(test_df.columns)}")
print("\nColumn descriptions:")
print("- row_id: Unique identifier for each prediction row")
print("- item_id: Location identifier (Location4)")
print("- anchor_time: The reference time point")
print("- Time: The target prediction time (anchor_time + 24h)")
print("- anchor_*: Weather covariates at the anchor time")

In [ ]:
# Focus on test dataset for detailed analysis
dataset_df = test_df.copy()

# Analyze dataset
dataset_head = dataset_df.head().to_string()
dataset_tails = dataset_df.tail().to_string()
dataset_shape = dataset_df.shape
rows, cols = dataset_shape

# Inspect and log head, tail, dimensions and shape
print(f"Dataset head:\n{dataset_head}")
print(f"\nDataset tail:\n{dataset_tails}")
print(f"\nDataset shape: {dataset_shape}")
print(f"Dataset has {cols} columns and {rows} rows")

In [ ]:
# Ensure all columns are shown when printing
pd.set_option('display.max_columns', None)

# Summary statistics of numeric columns
numeric_cols = dataset_df.select_dtypes(include=[np.number]).columns
print("Summary statistics:\n", dataset_df[numeric_cols].describe())
print("")
print("")

# Check for missing data
missing_values = dataset_df.isnull().sum()
print(f"Missing values per column:\n{missing_values}")
print("")
print("")

# Check for duplicate rows
duplicate_rows = dataset_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_rows}")

## 2. Data Type Analysis and Time Column Processing

In [ ]:
# Check data types
print("Data types:")
print(dataset_df.dtypes)
print("\n")

# Convert Time columns to datetime
dataset_df['anchor_time'] = pd.to_datetime(dataset_df['anchor_time'])
dataset_df['Time'] = pd.to_datetime(dataset_df['Time'])
print(f"Anchor time range: {dataset_df['anchor_time'].min()} to {dataset_df['anchor_time'].max()}")
print(f"Target time range: {dataset_df['Time'].min()} to {dataset_df['Time'].max()}")

# Verify the 24-hour prediction window
time_diff = (dataset_df['Time'] - dataset_df['anchor_time']).dt.total_seconds() / 3600
print(f"\nTime difference between anchor and target (hours): {time_diff.unique()}")

# Check unique locations
print(f"\nUnique locations: {dataset_df['item_id'].unique()}")
print(f"Number of unique locations: {dataset_df['item_id'].nunique()}")

## 3. Row ID Analysis

In [ ]:
# Analyze row_id structure
print("Sample row_id values:")
print(dataset_df['row_id'].head(10).tolist())
print("\nRow ID structure analysis:")
print("Format appears to be: Location4_YYYYMMDDHHMM_T24")
print("Where:")
print("- Location4: The location identifier")
print("- YYYYMMDDHHMM: Timestamp in format (anchor_time)")
print("- T24: Indicates 24-hour ahead prediction")

# Extract timestamp from row_id and verify consistency
def extract_timestamp_from_row_id(row_id):
    parts = row_id.split('_')
    if len(parts) >= 2:
        timestamp_str = parts[1]
        return pd.to_datetime(timestamp_str, format='%Y%m%d%H%M')
    return None

dataset_df['extracted_time'] = dataset_df['row_id'].apply(extract_timestamp_from_row_id)
time_consistency = (dataset_df['extracted_time'] == dataset_df['anchor_time']).all()
print(f"\nTime consistency between row_id and anchor_time: {time_consistency}")

## 4. Weather Variables Analysis

In [ ]:
# Get weather columns (anchor_* columns)
weather_columns = [col for col in dataset_df.columns if col.startswith('anchor_') and dataset_df[col].dtype in ['int64', 'float64']]
print(f"Weather variables: {weather_columns}")
print(f"Number of weather variables: {len(weather_columns)}")

# Create a mapping of weather variables to their descriptions
weather_descriptions = {
    'anchor_temperature_2m': 'Temperature at 2m height (°F)',
    'anchor_relativehumidity_2m': 'Relative humidity at 2m height (%)',
    'anchor_dewpoint_2m': 'Dew point at 2m height (°F)',
    'anchor_windspeed_10m': 'Wind speed at 10m height (mph)',
    'anchor_windspeed_100m': 'Wind speed at 100m height (mph)',
    'anchor_winddirection_10m': 'Wind direction at 10m height (degrees)',
    'anchor_winddirection_100m': 'Wind direction at 100m height (degrees)',
    'anchor_windgusts_10m': 'Wind gusts at 10m height (mph)'
}

print("\nWeather variable descriptions:")
for var, desc in weather_descriptions.items():
    print(f"- {var}: {desc}")

## 5. Test vs Training Data Comparison (from analyze_data.py)

This section replicates the comparison analysis from analyze_data.py to understand how test data relates to training data.

In [ ]:
# Load training data for comparison
train_df = pd.read_csv('../data/train.csv')
train_df['Time'] = pd.to_datetime(train_df['Time'])

# Filter Location4 from training data
loc4_train = train_df[train_df['item_id'] == 'Location4']

print(f"Training data shape: {train_df.shape}")
print(f"Location4 training data shape: {loc4_train.shape}")
print(f"Test data shape: {dataset_df.shape}")

In [ ]:
# Test data anchor weather vs training weather comparison (matching analyze_data.py)
print("Test data anchor weather vs training weather comparison:")
print(f"Test anchor wind speeds (10m): {dataset_df['anchor_windspeed_10m'].describe()}")
print(f"\nTraining wind speeds (10m): {loc4_train['windspeed_10m'].describe()}")

print(f"\nTest anchor wind speeds (100m): {dataset_df['anchor_windspeed_100m'].describe()}")
print(f"\nTraining wind speeds (100m): {loc4_train['windspeed_100m'].describe()}")

In [ ]:
# Detailed comparison of all weather variables
print("\nDetailed Weather Variable Comparison:")
print("=" * 50)

# Map anchor columns to training columns
comparison_mapping = {
    'anchor_temperature_2m': 'temperature_2m',
    'anchor_relativehumidity_2m': 'relativehumidity_2m',
    'anchor_dewpoint_2m': 'dewpoint_2m',
    'anchor_windspeed_10m': 'windspeed_10m',
    'anchor_windspeed_100m': 'windspeed_100m',
    'anchor_winddirection_10m': 'winddirection_10m',
    'anchor_winddirection_100m': 'winddirection_100m',
    'anchor_windgusts_10m': 'windgusts_10m'
}

for test_col, train_col in comparison_mapping.items():
    print(f"\n{test_col.replace('anchor_', '').upper()}:")
    print(f"  Test (anchor) - Mean: {dataset_df[test_col].mean():.2f}, Std: {dataset_df[test_col].std():.2f}")
    print(f"  Train        - Mean: {loc4_train[train_col].mean():.2f}, Std: {loc4_train[train_col].std():.2f}")
    print(f"  Difference   - Mean: {dataset_df[test_col].mean() - loc4_train[train_col].mean():.2f}")

In [ ]:
# Create comparison visualizations
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for i, (test_col, train_col) in enumerate(comparison_mapping.items()):
    # Create histograms for comparison
    axes[i].hist(loc4_train[train_col], bins=20, alpha=0.7, label='Training', color='blue', density=True)
    axes[i].hist(dataset_df[test_col], bins=20, alpha=0.7, label='Test (anchor)', color='red', density=True)
    axes[i].set_title(f'{test_col.replace("anchor_", "").replace("_", " ").title()}')
    axes[i].set_xlabel('Value')
    axes[i].set_ylabel('Density')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{test_graphs_dir}/test_vs_train_weather_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
print(f"Saved: {test_graphs_dir}/test_vs_train_weather_comparison.png")

In [ ]:
# Temporal comparison - show when test period occurs relative to training
print("\nTemporal Analysis:")
print(f"Training period: {loc4_train['Time'].min()} to {loc4_train['Time'].max()}")
print(f"Test anchor period: {dataset_df['anchor_time'].min()} to {dataset_df['anchor_time'].max()}")
print(f"Test target period: {dataset_df['Time'].min()} to {dataset_df['Time'].max()}")

# Check if there's any overlap
train_end = loc4_train['Time'].max()
test_start = dataset_df['anchor_time'].min()
gap_days = (test_start - train_end).days
print(f"\nGap between training end and test start: {gap_days} days")

## 6. Time-Based Visualizations

In [ ]:
# Create time series plots for all weather variables
print(f"Creating time series plots for {len(weather_columns)} weather variables")

# Create subplot grid - use 3x3 to ensure we have enough subplots
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
axes = axes.flatten()

for i, col in enumerate(weather_columns):
    if i < len(axes):  # Safety check
        axes[i].plot(dataset_df['anchor_time'], dataset_df[col], marker='o', linewidth=2, markersize=4)
        axes[i].set_title(f'{col} over Time', fontsize=12, fontweight='bold')
        axes[i].set_xlabel('Anchor Time')
        axes[i].set_ylabel(weather_descriptions.get(col, col))
        axes[i].grid(True, alpha=0.3)
        axes[i].tick_params(axis='x', rotation=45)

# Hide unused subplots
for i in range(len(weather_columns), len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.savefig(f'{test_graphs_dir}/weather_variables_timeseries.png', dpi=300, bbox_inches='tight')
plt.show()
print(f"Saved: {test_graphs_dir}/weather_variables_timeseries.png")

## 7. Distribution Analysis

In [ ]:
# Create distribution plots for all weather variables
print(f"Creating distribution plots for {len(weather_columns)} weather variables")

# Create subplot grid - use 3x3 to ensure we have enough subplots
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
axes = axes.flatten()

for i, col in enumerate(weather_columns):
    if i < len(axes):  # Safety check
        # Histogram
        axes[i].hist(dataset_df[col], bins=15, alpha=0.7, color='skyblue', edgecolor='black')
        axes[i].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
        axes[i].set_xlabel(weather_descriptions.get(col, col))
        axes[i].set_ylabel('Frequency')
        axes[i].grid(True, alpha=0.3)
        
        # Add statistics text
        mean_val = dataset_df[col].mean()
        std_val = dataset_df[col].std()
        axes[i].axvline(mean_val, color='red', linestyle='--', alpha=0.8, label=f'Mean: {mean_val:.2f}')
        axes[i].legend()

# Hide unused subplots
for i in range(len(weather_columns), len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.savefig(f'{test_graphs_dir}/weather_variables_distributions.png', dpi=300, bbox_inches='tight')
plt.show()
print(f"Saved: {test_graphs_dir}/weather_variables_distributions.png")

## 8. Correlation Analysis

In [ ]:
# Calculate correlation matrix for weather variables
weather_data = dataset_df[weather_columns]
correlation_matrix = weather_data.corr()

# Create correlation heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .8}, fmt='.3f')
plt.title('Correlation Matrix of Weather Variables', fontsize=16, fontweight='bold', pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(f'{test_graphs_dir}/weather_correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()
print(f"Saved: {test_graphs_dir}/weather_correlation_matrix.png")

# Print high correlations
print("\nHigh correlations (|r| > 0.7):")
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_val = correlation_matrix.iloc[i, j]
        if abs(corr_val) > 0.7:
            print(f"{correlation_matrix.columns[i]} vs {correlation_matrix.columns[j]}: {corr_val:.3f}")

## 9. Wind Analysis

In [ ]:
# Wind speed comparison (10m vs 100m)
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(dataset_df['anchor_windspeed_10m'], dataset_df['anchor_windspeed_100m'], alpha=0.7)
plt.xlabel('Wind Speed at 10m (mph)')
plt.ylabel('Wind Speed at 100m (mph)')
plt.title('Wind Speed: 10m vs 100m Height')
plt.grid(True, alpha=0.3)

# Add diagonal line for reference
min_val = min(dataset_df['anchor_windspeed_10m'].min(), dataset_df['anchor_windspeed_100m'].min())
max_val = max(dataset_df['anchor_windspeed_10m'].max(), dataset_df['anchor_windspeed_100m'].max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.5, label='Equal speeds')
plt.legend()

# Wind direction comparison (10m vs 100m)
plt.subplot(1, 2, 2)
plt.scatter(dataset_df['anchor_winddirection_10m'], dataset_df['anchor_winddirection_100m'], alpha=0.7)
plt.xlabel('Wind Direction at 10m (degrees)')
plt.ylabel('Wind Direction at 100m (degrees)')
plt.title('Wind Direction: 10m vs 100m Height')
plt.grid(True, alpha=0.3)

# Add diagonal line for reference
plt.plot([0, 360], [0, 360], 'r--', alpha=0.5, label='Equal directions')
plt.legend()

plt.tight_layout()
plt.savefig(f'{test_graphs_dir}/wind_analysis.png', dpi=300, bbox_inches='tight')
plt.show()
print(f"Saved: {test_graphs_dir}/wind_analysis.png")

## 10. Temporal Patterns

In [ ]:
# Extract temporal features
dataset_df['month'] = dataset_df['anchor_time'].dt.month
dataset_df['day_of_week'] = dataset_df['anchor_time'].dt.dayofweek
dataset_df['hour'] = dataset_df['anchor_time'].dt.hour

# Create temporal analysis plots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Monthly patterns for temperature
monthly_temp = dataset_df.groupby('month')['anchor_temperature_2m'].agg(['mean', 'std']).reset_index()
axes[0, 0].errorbar(monthly_temp['month'], monthly_temp['mean'], yerr=monthly_temp['std'], 
                   marker='o', capsize=5, capthick=2)
axes[0, 0].set_title('Monthly Temperature Patterns')
axes[0, 0].set_xlabel('Month')
axes[0, 0].set_ylabel('Temperature (°F)')
axes[0, 0].grid(True, alpha=0.3)

# Daily patterns for wind speed
daily_wind = dataset_df.groupby('day_of_week')['anchor_windspeed_10m'].agg(['mean', 'std']).reset_index()
day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
axes[0, 1].errorbar(daily_wind['day_of_week'], daily_wind['mean'], yerr=daily_wind['std'], 
                   marker='o', capsize=5, capthick=2)
axes[0, 1].set_title('Daily Wind Speed Patterns')
axes[0, 1].set_xlabel('Day of Week')
axes[0, 1].set_ylabel('Wind Speed (mph)')
axes[0, 1].set_xticks(range(7))
axes[0, 1].set_xticklabels(day_names)
axes[0, 1].grid(True, alpha=0.3)

# Hourly patterns (all data is at 12:00, so this will be uniform)
hourly_temp = dataset_df.groupby('hour')['anchor_temperature_2m'].agg(['mean', 'count']).reset_index()
axes[1, 0].bar(hourly_temp['hour'], hourly_temp['count'])
axes[1, 0].set_title('Hourly Distribution of Data Points')
axes[1, 0].set_xlabel('Hour of Day')
axes[1, 0].set_ylabel('Count')
axes[1, 0].grid(True, alpha=0.3)

# Time series of all anchor times
axes[1, 1].plot(dataset_df['anchor_time'], dataset_df['anchor_temperature_2m'], 'o-', alpha=0.7)
axes[1, 1].set_title('Temperature Time Series')
axes[1, 1].set_xlabel('Anchor Time')
axes[1, 1].set_ylabel('Temperature (°F)')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{test_graphs_dir}/temporal_patterns.png', dpi=300, bbox_inches='tight')
plt.show()
print(f"Saved: {test_graphs_dir}/temporal_patterns.png")

## 11. Statistical Summary and Insights

In [ ]:
# Comprehensive statistical summary
# Force correct weather_columns definition to avoid datetime errors
weather_columns = [col for col in dataset_df.columns if col.startswith('anchor_') and dataset_df[col].dtype in ['int64', 'float64']]
print("=== TEST DATASET STATISTICAL SUMMARY ===")
print(f"Total number of predictions required: {len(dataset_df)}")
print(f"Prediction period: {dataset_df['anchor_time'].min().strftime('%Y-%m-%d')} to {dataset_df['anchor_time'].max().strftime('%Y-%m-%d')}")
print(f"All predictions are for Location: {dataset_df['item_id'].unique()[0]}")
print(f"Prediction horizon: 24 hours ahead")
print(f"All anchor times are at: {dataset_df['hour'].unique()[0]}:00 (noon)")

print("\n=== WEATHER VARIABLE RANGES ===")
# Filter to only numeric weather columns to avoid datetime formatting errors
numeric_weather_cols = [col for col in weather_columns if dataset_df[col].dtype in ['int64', 'float64']]
print(f"Analyzing {len(numeric_weather_cols)} numeric weather variables")
for col in numeric_weather_cols:
    min_val = dataset_df[col].min()
    max_val = dataset_df[col].max()
    mean_val = dataset_df[col].mean()
    std_val = dataset_df[col].std()
    print(f"{col}:")
    print(f"  Range: {min_val:.2f} to {max_val:.2f}")
    print(f"  Mean ± Std: {mean_val:.2f} ± {std_val:.2f}")

print("\n=== DATA QUALITY ASSESSMENT ===")
print(f"Missing values: {dataset_df.isnull().sum().sum()}")
print(f"Duplicate rows: {dataset_df.duplicated().sum()}")
print(f"Data completeness: {(1 - dataset_df.isnull().sum().sum() / (len(dataset_df) * len(dataset_df.columns))) * 100:.1f}%")

print("\n=== KEY INSIGHTS ===")
print("1. This is a next-day power prediction task for Location4")
print("2. All predictions are made at noon (12:00) for the next day at noon")
print("3. Weather covariates are provided at the anchor time only")
print("4. The dataset covers approximately 2 months of predictions")
print("5. No missing values - clean dataset ready for modeling")
print("6. Wind speeds are consistently higher at 100m than at 10m height")
print("7. Wind directions at different heights are highly correlated")

## 12. Comparison with Training Data Context

In [ ]:
# Load training data for comparison
try:
    train_df = pd.read_csv('../data/train.csv')
    train_df['Time'] = pd.to_datetime(train_df['Time'])
    
    print("=== COMPARISON WITH TRAINING DATA ===")
    print(f"Training data period: {train_df['Time'].min().strftime('%Y-%m-%d')} to {train_df['Time'].max().strftime('%Y-%m-%d')}")
    print(f"Test data period: {dataset_df['anchor_time'].min().strftime('%Y-%m-%d')} to {dataset_df['anchor_time'].max().strftime('%Y-%m-%d')}")
    
    # Check if test period follows training period
    train_end = train_df['Time'].max()
    test_start = dataset_df['anchor_time'].min()
    
    print(f"\nTime gap between training end and test start: {(test_start - train_end).days} days")
    
    # Location4 data in training set
    loc4_train = train_df[train_df['item_id'] == 'Location4']
    print(f"\nLocation4 in training data:")
    print(f"  Records: {len(loc4_train)}")
    print(f"  Period: {loc4_train['Time'].min().strftime('%Y-%m-%d')} to {loc4_train['Time'].max().strftime('%Y-%m-%d')}")
    print(f"  Power range: {loc4_train['Power'].min():.4f} to {loc4_train['Power'].max():.4f}")
    
except FileNotFoundError:
    print("Training data not found for comparison")

## 13. Feature Engineering Suggestions

In [ ]:
print("=== FEATURE ENGINEERING SUGGESTIONS ===")
print("\nBased on the test data analysis, consider these features:")
print("\n1. TEMPORAL FEATURES:")
print("   - Month, day of year, day of week")
print("   - Season indicators")
print("   - Days since/until solstices and equinoxes")

print("\n2. WEATHER INTERACTIONS:")
print("   - Wind power proxy: windspeed^3 (cube of wind speed)")
print("   - Temperature-humidity interactions")
print("   - Wind shear: difference between 100m and 10m speeds")
print("   - Wind direction consistency between heights")

print("\n3. LAGGED FEATURES (from training data):")
print("   - Previous day's power output")
print("   - Rolling averages of weather variables (3, 7, 30 days)")
print("   - Weather change rates (day-to-day differences)")

print("\n4. DERIVED WEATHER FEATURES:")
print("   - Heat index (temperature + humidity)")
print("   - Wind chill factor")
print("   - Atmospheric pressure proxy (temperature + humidity)")
print("   - Wind direction in radians (sin/cos transformation)")

print("\n5. POWER-SPECIFIC FEATURES:")
print("   - Optimal wind speed ranges for power generation")
print("   - Temperature efficiency factors")
print("   - Seasonal power generation patterns")

# Demonstrate some feature engineering
print("\n=== EXAMPLE FEATURE ENGINEERING ===")
dataset_df['wind_power_proxy'] = dataset_df['anchor_windspeed_100m'] ** 3
dataset_df['wind_shear'] = dataset_df['anchor_windspeed_100m'] - dataset_df['anchor_windspeed_10m']
dataset_df['temp_humidity_interaction'] = dataset_df['anchor_temperature_2m'] * dataset_df['anchor_relativehumidity_2m']
dataset_df['wind_direction_consistency'] = abs(dataset_df['anchor_winddirection_100m'] - dataset_df['anchor_winddirection_10m'])

print("Created example features:")
print(f"- wind_power_proxy: mean = {dataset_df['wind_power_proxy'].mean():.2f}")
print(f"- wind_shear: mean = {dataset_df['wind_shear'].mean():.2f}")
print(f"- temp_humidity_interaction: mean = {dataset_df['temp_humidity_interaction'].mean():.2f}")
print(f"- wind_direction_consistency: mean = {dataset_df['wind_direction_consistency'].mean():.2f}")